In [29]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
#Normalisation
b=Normalizer()
data=pd.read_excel('/content/default of credit card clients.xls')
#Imputation in place of feature engineering and data preprocessing
imputer= SimpleImputer(missing_values=np.nan,strategy='mean')
A=data.iloc[1:,:-1]
B=pd.DataFrame(map(int,data.iloc[1:,-1]))
A=imputer.fit_transform(A)
A=b.transform(A)
X_r,Y_r = (A, B)
#SMOTE for class imbalance
X_resampled, Y_resampled = SMOTE().fit_resample(X_r, Y_r)
#Data splitting into test and train
X_train,X_test,Y_train,Y_test = train_test_split(X_resampled,Y_resampled,test_size=0.2)
#Random Forest
k=RandomForestClassifier()
k.fit(X_train,np.array(Y_train))
y_preprf=k.predict(X_test)
accuracyrf=accuracy_score(Y_test,y_preprf)
print(accuracyrf)
#Logistic Regression
g=LogisticRegression()
g.fit(X_train,np.array(Y_train))
y_preplr=g.predict(X_test)
accuracylr=accuracy_score(Y_test,y_preplr)
print(accuracylr)
#Support Vector Machine
model = SVC(kernel='linear')
model.fit(X_train, np.array(Y_train))
y_prepsv=model.predict(X_test)
accuracysv=accuracy_score(Y_test,y_prepsv)
print(accuracysv)
#XGBoost
dtrain_reg = xgb.DMatrix(X_train, Y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, Y_test, enable_categorical=True)
params={"Objective":"binary:logistic"} #"tree_method": "gpu_hist" ,"num_class": 5}
#XGB model generally used for further classification
model2 = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=10,
)
#XGB model used ultimately
model3=xgb.XGBClassifier()
model3.fit(X_train,Y_train)
y_prepxg=model3.predict(X_test)
ab=Y_test
accuracyxg=accuracy_score(ab,y_prepxg)
print(accuracyxg)
#Confusion Matrix
def cm(Y_train,y_test, y_pred):
  conmat = confusion_matrix(y_test, y_pred)
  val = np.mat(conmat)
  classnames = list(Y_train)
  df_cm = pd.DataFrame(val)
  print(df_cm)
cm(Y_train,Y_test,y_preplr)
cm(Y_train,Y_test,y_preprf)
cm(Y_train,Y_test,y_prepsv)
cm(Y_train,Y_test,y_prepxg)
#Use of Confusion Matrix and accuracy score signifies that use of Random Forest
#is most beneficial

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.8672159212497325


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6201583565161567


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6048576931307511


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:36:51] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "Objective" } are not used.

  warnings.warn(smsg, UserWarning)


0.8340466509736786
      0     1
0  2834  1856
1  1694  2962
      0     1
0  4116   574
1   667  3989
      0     1
0  3011  1679
1  2014  2642
      0     1
0  4080   610
1   941  3715


First of all , an appropiate database containing variable names and non-biased data was found and used.
Normalisation and imputation is implemented.Imputation method seemed better than using the other methods to remove the empty values(though there wasn't any such missing value in the database).The used of feature engineering isn't beneficial as the data used can't be further broke down, insetead it might degrade our model, thus its not implemented.Encoding to categorial variables was also not required as it was already implemented.Then SMOTE was implemented as it seemed to be the best under the situation of our model to take care of class imbalance.Then Logical Regression, Random Forests, XGBoost and SVM algorithms are implemented.Each model was evaluated using the accuracy score and confusion matrices. The comparison showed that the Random Forest method suits the best to our model.As the class imbalance was already implemented, use of precision-recall score, according to me, is reduntant and observable in the confusion Matrices. Also, Lightgbm was not implemented as it would be quite time consuming, and that our data isn't skewed enough to make it necessary to use it, as we didn't implement further feature and model evaluations.
Results-
Accuracy scores-
RF-0.867
LR-0.62
SVM- 0.60
XGBoost-0.834
Confusion Matrices-
LR-
    0     1
0  2834  1856
1  1694  2962
RF-
     0     1
0  4116   574
1   667  3989
SVM-
      0     1
0  3011  1679
1  2014  2642
XGB-
      0     1
0  4080   610
1   941  3715
**Thus Random Forest method seems the most appropiate